# Difftest Results

Connect to results database:

In [ ]:
import db
from db import *
%run util.py

hostname = "cc1"
db.init(hostname)

## Overview

In [ ]:
import pandas as pd

session = db.make_session()

TABLE_NAMES = ["CLSmith", "CLSmith w. cldrive", "GitHub", "CLgen", "CLgen w. cl_launcher", "CLgen w. co"]
TABLES = [CLSmithResult, cldriveCLSmithResult, GitHubResult, CLgenResult, cl_launcherCLgenResult, coCLgenResult]

data = [
    ("#. Programs", [session.query(t.program_id).group_by(t.program_id).count() for t in TABLES]),
    ("#. Testbeds", [session.query(t.testbed_id).group_by(t.testbed_id).count() for t in TABLES]),
    ("#. Params", [session.query(t.params_id).group_by(t.params_id).count() for t in TABLES]),
    ("#. Results", [session.query(t).count() for t in TABLES]),
]
i, d = zip(*data)
overview = pd.DataFrame(list(d), index=i, columns=TABLE_NAMES)
overview

# Experimental Setup

### TestBeds

A testbed is a combination of host platform and OpenCL device.

In [ ]:
import sqlalchemy as sql

q = session.query(Testbed).order_by(sql.func.field(Testbed.devtype, 'GPU', 'CPU', 'Emulator'))

data = []
for testbed in q:
    data.append(
        (testbed.id, [
            host_str(testbed.host), device_str(testbed.device),
            DRIVERS.get(testbed.driver, testbed.driver), testbed.opencl, testbed.devtype] +
         [session.query(t.testbed).filter(t.testbed == testbed).count() for t in TABLES]))
i, d = zip(*data)
testbeds = pd.DataFrame(list(d), index=i, columns=["Operating System", "Device", "Driver", "OpenCL", "Device type"] + [f"#. {t}" for t in TABLE_NAMES])

if len(CONFIGURATIONS) != session.query(Testbed).count():
    import sys
    print("warning: missing testbed(s)", file=sys.stderr)
testbeds

In [ ]:
# push LaTex to Overleaf
!cd ~/docs/paper-project_b/ && git pull --rebase

import os
from collections import OrderedDict

def get_total_submitted(testbed: Testbed):
    submitable_results = [coCLgenResult, cl_launcherCLgenResult]
    
    def get_submitted(table):
        return session.query(table).filter(table.testbed_id == testbed.id, table.submitted).count()

    def get_generated(table):
        return session.query(table).filter(table.testbed_id == testbed.id, sql.or_(table.submitted, table.dupe)).count()
    
    return (
        sum(get_generated(table) for table in submitable_results), 
        sum(get_submitted(table) for table in submitable_results)
    )


def get_testbed_info(config_id, testbed_id):
    testbed = session.query(Testbed).filter(Testbed.id == testbed_id).first()
    d = OrderedDict()
    d["#."] = config_id
    d["Device"] = testbed.device
    d["Platform"] = platform_str(testbed.platform)
    d["Driver"] = driver_str(testbed.driver)
    d["OpenCL"] = testbed.opencl
    d["Operating system"] = host_str(testbed.host)
    d["Device type"] = devtype_str(testbed.devtype)
    d["B.R. Generated"], d["B.R. Submitted"] = get_total_submitted(testbed)
    return d

table = pd.DataFrame([get_testbed_info(*x) for x in CONFIGURATIONS])
with open(os.path.expanduser("~/docs/paper-project_b/build/tab/platforms.tex"), "w") as outfile:
    table.to_latex(buf=outfile, index=None)
!cd ~/docs/paper-project_b/build && git add . && git commit -m "auto: build/tab/platforms.tex" && git push
table

In [ ]:
import subprocess
import sys

from labm8 import fs

null_columns = " & ".join(["-"] * 6)

# Measured average CLgen sample time
clgen_generation_time = .5

def get_counts(results_table, params_table, testbed, no_opt: bool, time_limit: int=None):
    """
    Return results for the specified time limit
    """
    # If no time limit is provided, return all results.
    if time_limit is None:
        return {
            "w": q.filter(results_table.classification == "w").count(),
            "bf": q.filter(results_table.classification == "bf").count(),
            "c": q.filter(results_table.classification == "c").count(),
            "to": q.filter(results_table.classification == "to").count(),
            "pass": q.filter(results_table.classification == "pass").count(),
            "fail": q.filter(results_table.classification == "fail").count(),
            "total": q.count()
        }
    
    # Otherwise, simulate results for a fixed period of time
    t = 0  # elapsed time
    counts = {None: 0, "w": 0, "bf": 0, "c": 0, "to": 0, "pass": 0, "fail": 0}

    param_ids = session.query(params_table.id).filter(params_table.optimizations == no_opt)
    q = session.query(results_table).filter(
        results_table.testbed_id == testbed.id, 
        results_table.params_id.in_(param_ids))
    
    for result in q.order_by(results_table.id):
        if hasattr(result.program, 'runtime'):
            generation_time = result.program.runtime
        else:
            generation_time = clgen_generation_time
        exec_time = generation_time + result.runtime
        if t + exec_time > time_limit:
            hours = t / 60 / 60
            break
        t += exec_time
        counts[result.classification] = counts.get(result.classification, 0) + 1
    else:
        counts['total'] = '*'  # asterisk denotes insufficient results to reach time limit

    # remove unclassified results from table
    counts.pop(None)
        
    # sum up all results
    total = counts.pop("total", "")
    counts['total'] = str(sum(counts.values())) + total

    return counts


def get_columns(results_table, params_table, testbed: Testbed, no_opt: bool, time_limit: int=172800):
    print(f"{results_table.__tablename__} {testbed.device} {no_opt} ", end="")
    c = get_counts(results_table, params_table, testbed, no_opt, time_limit)
    print(c)
    sys.stdout.flush()
    if c['total'].endswith('*'):
        print(f"insufficient {results_table.__tablename__} for {testbed.device} {no_opt}", file=sys.stderr)
        sys.stderr.flush()
    return f"{c['w']} & {c['bf']} & {c['c']} & {c['to']} & {c['pass']} & {c['total']}"


def get_row(config_id, testbed_id):
    """ get mega-table row """
    testbed = session.query(Testbed).filter(Testbed.id == testbed_id).first()
    platform_name = platform_str(testbed.platform)
    device_name = device_str(testbed.device)
    driver_name = driver_str(testbed.driver)
    
    time_limit = 172800
    
    noopt = False
    clsmith_columns = get_columns(CLSmithResult, cl_launcherParams, testbed, False)
    # clsmith_columns = null_columns
    clgen_columns = get_columns(CLgenResult, cldriveParams, testbed, noopt, time_limit)
    
    noopt = True
    clsmith_noopt_columns = get_columns(CLSmithResult, cl_launcherParams, testbed, True)
    # clsmith_noopt_columns = null_columns
    clgen_noopt_columns = get_columns(CLgenResult, cldriveParams, testbed, noopt, time_limit)
    
    return f"""\\multirow{{ 2}}{{*}}{{{config_id}}} & \\multirow{{ 2}}{{*}}{{{platform_name}}} & \
\\multirow{{ 2}}{{*}}{{{device_name}}} & \
$-$ & {clsmith_columns}       & {clgen_columns} \\\\
& & & \
$+$ & {clsmith_noopt_columns} & {clgen_noopt_columns} \\\\"""

rows = "\n\\hline\n".join(get_row(*x) for x in CONFIGURATIONS)

latex = f"""\
\\begin{{tabular}}{{llll | rrrrrr | rrrrrr }}
  \\toprule
  & & & & \\multicolumn{{6}}{{c|}}{{\\textbf{{CLSmith}}}} & \\multicolumn{{6}}{{c}}{{\\textbf{{CLgen}}}} \\\\
  \\textbf{{\\#.}} & \\textbf{{Platform}} & \\textbf{{Device}} & $\\pm$ & 
  \\textbf{{w}} & \\textbf{{bf}} & \\textbf{{c}} & \\textbf{{to}} & \\cmark & \\textbf{{total}} & 
  \\textbf{{w}} & \\textbf{{bf}} & \\textbf{{c}} & \\textbf{{to}} & \\cmark & \\textbf{{total}} \\\\
  \\midrule
  {rows}
  \\bottomrule
\\end{{tabular}}
"""

# push LaTex to Overleaf
!cd ~/docs/paper-project_b/ && git pull --rebase

with open(os.path.expanduser("~/docs/paper-project_b/build/tab/megatable.tex"), "w") as outfile:
    print(latex, file=outfile)

!cd ~/docs/paper-project_b/build && git add . && git commit -m "auto: build/tab/megatable.tex" && git push

## Runtime Parameters

### cl_launcher

In [ ]:
CL_LAUNCHER_TABLE_NAMES = ["CLSmith", "CLgen w. cl_launcher"]
CL_LAUNCHER_TABLES = [CLSmithResult, cl_launcherCLgenResult]

q = session.query(cl_launcherParams).order_by(
        cl_launcherParams.gsize_x, cl_launcherParams.gsize_y, cl_launcherParams.gsize_z,
        cl_launcherParams.lsize_x, cl_launcherParams.lsize_y, cl_launcherParams.lsize_z,
        cl_launcherParams.optimizations)

data = []
for param in q:
    nresult_param = session.query(CLSmithResult).filter(CLSmithResult.params == param).count()
    data.append((
        param.id, [param.gsize, param.lsize, param.optimizations_on_off ] + [
            session.query(t).filter(t.params == param).count()
            for t in CL_LAUNCHER_TABLES
        ]))
i, d = zip(*data)

cl_launcher_params = pd.DataFrame(list(d), index=i, columns=[
    "Global size", "Local size", "Optimizations"] + [
        f"#. {t}" for t in CL_LAUNCHER_TABLE_NAMES])
cl_launcher_params

### cldrive

In [ ]:
CLDRIVE_TABLE_NAMES = ["CLSmith w. cldrive", "GitHub", "CLgen"]
CLDRIVE_TABLES = [cldriveCLSmithResult, GitHubResult, CLgenResult]

q = session.query(cldriveParams).order_by(
        cldriveParams.size,
        cldriveParams.gsize_x, cldriveParams.gsize_y, cldriveParams.gsize_z,
        cldriveParams.lsize_x, cldriveParams.lsize_y, cldriveParams.lsize_z,
        cldriveParams.generator, cldriveParams.scalar_val, cldriveParams.optimizations)

# push LaTex to Overleaf
!cd ~/docs/paper-project_b/ && git pull --rebase
data = []
for param in q:
    data.append([param.size, param.gsize, param.lsize, param.optimizations_on_off])
table = pd.DataFrame(data, index=range(1, len(data)+1), columns=[
    "Dataset Size", "Global size", "Workgroup size", "Optimizations"])
with open(os.path.expanduser("~/docs/paper-project_b/build/tab/cldrive-params.tex"), "w") as outfile:
    table.to_latex(buf=outfile)
!cd ~/docs/paper-project_b/build && git add . && git commit -m "auto: build/tab/cldrive-params.tex" && git push
table

# Experimental Results

## Runtimes

Excluding runs which terminated in non-zero status:

In [ ]:
import numpy as np

runtimes = [np.array(session.query(table.runtime).filter(table.status == 0).all()) for table in TABLES]
data = [
    ("Min", [r.min() for r in runtimes]),
    ("Median", [np.median(r) for r in runtimes]),
    ("Mean", [r.mean() for r in runtimes]),
    ("Max", [r.max() for r in runtimes])
]
i, d = zip(*data)
runtimes = pd.DataFrame(list(d), index=i, columns=TABLE_NAMES)
runtimes

## Outcomes & Classifications

**Pandas tables of outcomes**

In [ ]:
outcomes = {}

for name, table in zip(CL_LAUNCHER_TABLE_NAMES + CLDRIVE_TABLE_NAMES, CL_LAUNCHER_TABLES + CLDRIVE_TABLES):
    r = []
    for testbed in session.query(Testbed).all():
        nresult = session.query(table).filter(table.testbed == testbed).count()

        q = session.query(table.outcome, sql.func.count(table.outcome)).filter(
            table.testbed == testbed).group_by(table.outcome).order_by(
                sql.desc(sql.func.count(table.outcome)))

        for outcome, count in q.all():
            ratio = (count / nresult) * 100
            r.append((DEVICES.get(testbed.device, testbed.device), outcome, count, ratio))
    outcomes[name] = pd.DataFrame(r, columns=["Device", "Outcome", "Count", "% of Total Results"])

print("done.")

**Pandas tables of classifications**

In [ ]:
classifications = {}

classificationsSort = [
    'Invalid testcase',
    'Build failure',
    'Runtime crash',
    'No majority',
    'Wrong code',
    'Okay'
]

def escape(val):
    if val is None:
        return val
    else:
        return str(classificationsSort.index(val)) + ". " + val

for name, table in zip(CL_LAUNCHER_TABLE_NAMES + CLDRIVE_TABLE_NAMES, CL_LAUNCHER_TABLES + CLDRIVE_TABLES):
    r = []
    for testbed in session.query(Testbed).all():
        nresult = session.query(table).filter(table.testbed == testbed).count()

        q = session.query(table.classification, sql.func.count(table.classification)).filter(
            table.testbed == testbed).group_by(table.classification).order_by(
                sql.desc(sql.func.count(table.classification)))

        for val, count in q.all():
            ratio = (count / nresult) * 100
            r.append((DEVICES.get(testbed.device, testbed.device), escape(val), count, ratio))
    classifications[name] = pd.DataFrame(r, columns=["Device", "Classification", "Count", "% of Total Results"])

print("done.")

In [ ]:
classifications["CLgen w. cl_launcher"]

## Experimental Results

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from labm8 import viz
%matplotlib inline

def plot_outcomes(table, name, dictname=outcomes, key='Outcome'):
    ax = dictname[name].pivot('Device', key)['Count'].plot(
        kind='bar', stacked=True, colormap="Reds_r", sort_columns=True)

    nprog = session.query(table.program_id).group_by(table.program_id).count()
    nparam = session.query(table.params_id).group_by(table.params_id).count()
    plt.title(f"{nprog} {name} x {nparam} parameters")
    plt.ylabel("Results")
    plt.xlabel("")

    plt.ylim(0, nprog * nparam)

    # reverse legend order (because plot stacks from bottom to top, and legend goes from top to bottom)
    handles, labels = ax.get_legend_handles_labels()
    ax.legend(handles[::-1], labels[::-1], loc='center left', bbox_to_anchor=(1, 0.5))

    viz.finalise(figsize=(3.5, 8))
    
    
def summarize(table_name):
    """ summarize a table of classifications """
    table = classifications[table_name]

    def get_val(classification):
        try:
            return table.loc[
                (table['Device'] == device) & (table['Classification'] == classification)]['Count'].values[0]
        except IndexError:
            return 0
    
    columns = ['Platform', 'Device', 'Driver', 'Invalid Testcases', 'Build Failures', 'Runtime Crashes', 'Incorrect Outputs', 'Okay']
    devices = sorted(set(table['Device'].values))

    d = []    
    for device in devices:
        lookup = dict((v, k) for k, v in DEVICES.items())
        full_name = lookup.get(device, device)
        
        # lookup the testbed
        q = session.query(Testbed).filter(Testbed.device == full_name).all()
        if len(q) != 1:
            raise q
        testbed = q[0]
        
        r = [
            PLATFORMS.get(testbed.platform, testbed.platform),
            device,
            DRIVERS.get(testbed.driver, testbed.driver),
            get_val('0. Invalid testcase'),
            get_val('1. Build failure'),
            get_val('2. Runtime crash'),
            get_val('3. Wrong code'),
            get_val('4. Okay'),
        ]
        d.append(r)
    summary = pd.DataFrame(d, columns=columns, index=range(1, len(devices)+1))

    !cd ~/docs/paper-project_b/ && git pull --rebase >/dev/null
    name = '-'.join(table_name.split())
    with open(os.path.expanduser(f"~/docs/paper-project_b/build/tab/results-{name}.tex"), "w") as outfile:
        summary.to_latex(buf=outfile)
    !cd ~/docs/paper-project_b/build && git add . && git commit -m "auto: summarize table" >/dev/null && git push >/dev/null
    return summary

### CLSmith

In [ ]:
outcomes["CLSmith"]

In [ ]:
summarize('CLSmith')

In [ ]:
import numpy as np

def lc(src):
    return len(src.strip().split("\n"))

lcs = np.array([lc(row[0]) for row in session.query(CLSmithProgram.src)])

In [ ]:
np.median(lcs)

In [ ]:
plot_outcomes(CLSmithResult, "CLSmith", dictname=classifications, key='Classification')

### CLgen w. cl_launcher

In [ ]:
outcomes["CLgen w. cl_launcher"]

In [ ]:
summarize('CLgen w. cl_launcher')

In [ ]:
plot_outcomes(cl_launcherCLgenResult, "CLgen w. cl_launcher", dictname=classifications, key='Classification')

### CLSmith w. cldrive

In [ ]:
outcomes["CLSmith w. cldrive"]

In [ ]:
summarize('CLSmith w. cldrive')

In [ ]:
plot_outcomes(cldriveCLSmithResult, "CLSmith w. cldrive", dictname=classifications, key='Classification')

### GitHub

In [ ]:
outcomes["GitHub"]

In [ ]:
summarize('GitHub')

In [ ]:
plot_outcomes(GitHubResult, "GitHub", dictname=classifications, key='Classification')

### CLgen

In [ ]:
outcomes["CLgen"]

In [ ]:
summarize('CLgen')

In [ ]:
plot_outcomes(CLgenResult, "CLgen", dictname=classifications, key='Classification')